Load Scraped Data (Downlaod CSV File from "Submission\bayan_mohammad" Container\Folder)

In [0]:
# ---------------------------------------------------------
# LOAD REAL ESTATE & SELECT RELIABLE FEATURES
# ---------------------------------------------------------
import pandas as pd
import numpy as np

filename = "nyc_enriched_progress.csv"
print(f"Loading: {filename}")

# 1. Load Data
df_dirty = pd.read_csv(filename)

# 2. Selecting the relevant columns
# We keep 'sqft' for display, but won't use it for ML (since Airbnb lacks sqft)
keep_cols = ['clean_address', 'price', 'beds', 'baths', 'is_penthouse', 'is_luxury', 'sqft']
df_clean = df_dirty[keep_cols].copy()

# 3. Clean Price (Remove '$', ',')
df_clean['price'] = df_clean['price'].astype(str).str.replace(r'[$,]', '', regex=True)
df_clean['price'] = pd.to_numeric(df_clean['price'], errors='coerce')

# 4. Convert Booleans (TRUE/FALSE) to Integers (1/0)
bool_cols = ['is_penthouse', 'is_luxury']
for col in bool_cols:
    df_clean[col] = df_clean[col].astype(str).str.upper().map({
        'TRUE': 1, 'FALSE': 0, '1': 1, '0': 0, 'NAN': 0
    }).fillna(0)

# 5. Clean Numbers (Beds/Baths)
# Ensure they are floats, replace NaNs with defaults
df_clean['beds'] = pd.to_numeric(df_clean['beds'], errors='coerce').fillna(1)
df_clean['baths'] = pd.to_numeric(df_clean['baths'], errors='coerce').fillna(1)

# 6. Filter Invalid Rows
df_geo_input = df_clean.dropna(subset=['price', 'clean_address'])
df_geo_input = df_geo_input[df_geo_input['price'] > 10000] # Basic sanity check

print(f"Real Estate Data Ready: {len(df_geo_input)} rows.")
print("Features for Model:", ['beds', 'baths', 'is_penthouse', 'is_luxury'])
display(df_geo_input.head())

Loading: nyc_enriched_progress.csv
Real Estate Data Ready: 1592 rows.
Features for Model: ['beds', 'baths', 'is_penthouse', 'is_luxury']


clean_address,price,beds,baths,is_penthouse,is_luxury,sqft
"6 sutton square, New York, USA",1.9488E7,7.0,7.0,0,1,8534.0
"29 vandam street, 10014 manhattan, New York, USA",6500000.0,5.0,3.0,0,0,3200.0
"135 west 77th street, 10024 manhattan, New York, USA",1.1995E7,5.0,6.0,0,0,7200.0
"186-31 radnor road, 11432 jamaica estates, New York, USA",3100000.0,4.0,2.0,0,0,2750.0
"10 west 15th street, 10011 manhattan, New York, USA",849000.0,1.0,1.0,0,0,0.0


Geocoding

In [0]:
# ---------------------------------------------------------
# INSTALL REQUIRED LIBRARIES
# ---------------------------------------------------------
%pip install geopy plotly tqdm pandas scikit-learn

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# ---------------------------------------------------------
# GEOCODING (takes 10m Approximately)
# ---------------------------------------------------------
from geopy.geocoders import ArcGIS
from tqdm import tqdm

if 'lat' in df_geo_input.columns and df_geo_input['lat'].notnull().sum() > 0:
    print("Lat/Lon already exists. Skipping.")
    df_ready = df_geo_input.copy()
else:
    print(f"Geocoding {len(df_geo_input)} addresses...")
    geolocator = ArcGIS(user_agent="nyc_project_94290", timeout=10)
    
    lats, lons = [], []
    
    for addr in tqdm(df_geo_input['clean_address']):
        try:
            # Append City if missing for better accuracy
            query = addr if "New York" in str(addr) else f"{addr}, New York, NY"
            loc = geolocator.geocode(query)
            if loc:
                lats.append(loc.latitude)
                lons.append(loc.longitude)
            else:
                lats.append(None)
                lons.append(None)
        except:
            lats.append(None)
            lons.append(None)
            
    df_ready = df_geo_input.copy()
    df_ready['lat'] = lats
    df_ready['lng'] = lons
    
    # Drop rows where geocoding failed
    df_ready = df_ready.dropna(subset=['lat', 'lng'])
    print(f"Geocoding Complete. {len(df_ready)} locations found.")

Geocoding 1592 addresses...


 49%|████▉     | 779/1592 [05:30<05:44,  2.36i

*** WARNING: max output size exceeded, skipping output. ***

100%|██████████| 1592/1592 [11:18<00:00,  2.35it/s]

Geocoding Complete. 1592 locations found.


Load Airbnb & Engineer Matching Features

In [0]:
import nest_asyncio
import pandas as pd
import numpy as np
import os

# Apply patches
nest_asyncio.apply()

# --- 1. PROJECT SETTINGS ---
TARGET_CITY = "New York"
# We loop through price ranges to force the site to show us 3000+ items
PRICE_RANGES = [
    (0, 500000),
    (500000, 900000),
    (900000, 1500000),
    (1500000, 3000000),
    (3000000, 10000000)
]

# --- 2. AZURE STORAGE ---
# Replace with your storage account name
STORAGE_ACCOUNT = "lab94290" 
CONTAINER = "airbnb"
OUTPUT_PATH = f"abfss://{CONTAINER}@{STORAGE_ACCOUNT}.dfs.core.windows.net/student_submissions/nyc_arbitrage_final.csv"

# --- 3. BRIGHTDATA CREDENTIALS ---
BRIGHTDATA_USER = 'brd-customer-hl_80709a30-zone-scraping_browser_hostiq'
BRIGHTDATA_PASS = '3xihg62zyhi0' 
BRIGHTDATA_HOST = 'brd.superproxy.io:9222'

print(f"✅ Config Loaded. Targeting {TARGET_CITY} via Price Segmentation.")

✅ Config Loaded. Targeting New York via Price Segmentation.


In [0]:
# ---------------------------------------------------------
# LOAD AIRBNB & MATCH COLUMNS
# ---------------------------------------------------------
import re
from pyspark.sql.functions import col

# --- AZURE STORAGE AUTHENTICATION ---
storage_account = "lab94290"
container = "airbnb"
sas_token = "sp=rle&st=2025-12-24T17:37:04Z&se=2026-02-28T01:52:04Z&spr=https&sv=2024-11-04&sr=c&sig=a0lx%2BS6PuS%2FvJ9Tbt4NKdCJHLE9d1Y1D6vpE1WKFQtk%3D"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token.lstrip('?'))

# 1. Load Data
path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/airbnb_1_12_parquet"
df_spark = spark.read.parquet(path)
df_airbnb = df_spark.filter(col("location").contains("New York")).toPandas()

# --- Feature Engineering Functions ---

def parse_details(details_str):
    """Extracts beds/baths from Airbnb text to match your columns"""
    beds, baths = 1, 1
    try:
        if not isinstance(details_str, str): return 1, 1
        text = details_str.lower()
        
        bed_match = re.search(r'(\d+(\.\d+)?)\s*bed', text)
        bath_match = re.search(r'(\d+(\.\d+)?)\s*bath', text)
        
        if bed_match: beds = float(bed_match.group(1))
        if bath_match: baths = float(bath_match.group(1))
        return beds, baths
    except: return 1, 1

def check_attributes(row):
    """
    Creates 'is_luxury' and 'is_penthouse' flags for Airbnb 
    by scanning its text data (name/amenities).
    """
    # Combine text fields for scanning
    text = (str(row.get('name', '')) + " " + str(row.get('amenities', ''))).lower()
    
    # Logic for Penthouse
    is_penthouse = 1 if 'penthouse' in text else 0
    
    # Logic for Luxury (Doorman, Gym, Pool, View)
    luxury_keywords = ['doorman', 'gym', 'pool', 'view', 'concierge', 'hot tub']
    is_luxury = 1 if any(k in text for k in luxury_keywords) else 0
    
    return is_penthouse, is_luxury

print("Engineering Airbnb columns to match Real Estate data...")

# 1. Extract Beds/Baths
parsed = df_airbnb['details'].apply(parse_details)
df_airbnb['beds'] = [x[0] for x in parsed]
df_airbnb['baths'] = [x[1] for x in parsed]

# 2. Extract Flags (Luxury/Penthouse)
flags = df_airbnb.apply(check_attributes, axis=1)
df_airbnb['is_penthouse'] = [x[0] for x in flags]
df_airbnb['is_luxury'] = [x[1] for x in flags]

# 3. Clean Price
df_airbnb['price'] = df_airbnb['price'].astype(str).str.replace(r'[$,]', '', regex=True)
df_airbnb['price'] = pd.to_numeric(df_airbnb['price'], errors='coerce')

# 4. Standardize Location
if 'long' in df_airbnb.columns: df_airbnb.rename(columns={'long': 'lng'}, inplace=True)

# 5. Final Filter
df_train = df_airbnb.dropna(subset=['price', 'lat', 'lng', 'beds'])
df_train = df_train[df_train['price'] < 5000] # Remove outliers

print(f"Ground Truth Ready: {len(df_train)} listings.")
print("Matching Columns Created: ['beds', 'baths', 'is_penthouse', 'is_luxury']")
display(df_train[['price', 'beds', 'baths', 'is_penthouse', 'is_luxury']].head())

Engineering Airbnb columns to match Real Estate data...
Ground Truth Ready: 26889 listings.
Matching Columns Created: ['beds', 'baths', 'is_penthouse', 'is_luxury']


price,beds,baths,is_penthouse,is_luxury
100.0,1.0,1.0,0,0
135.0,3.0,2.0,0,0
113.0,1.0,1.0,0,0
349.0,2.0,1.0,0,1
750.0,4.0,4.5,0,1


Train & Predict

In [0]:
# ---------------------------------------------------------
# TRAIN MODEL & PREDICT
# ---------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np

# --- 1. CLEAN TRAINING DATA (Remove Outliers) ---
# Cap training prices at $2,000/night. Anything higher is likely an outlier or event pricing.
df_train_clean = df_train[df_train['price'] < 2000].copy()

# Features
features = ['lat', 'lng', 'beds', 'baths', 'is_luxury', 'is_penthouse']
print(f"Training on {len(df_train_clean)} listings (outliers removed)...")

X = df_train_clean[features]
y = df_train_clean['price']

# Train Model
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X, y)

# --- 2. PREDICT ---
# Prepare Real Estate Data
X_real_estate = df_ready[features].copy()
X_real_estate = X_real_estate.fillna(0)

# Predict Nightly Price
df_ready['predicted_nightly'] = model.predict(X_real_estate)

# --- 3. CALCULATE NET YIELD (The Realistic Formula) ---

# Conservative Assumptions
OCCUPANCY_RATE = 0.55       # 55% Occupancy (NYC average)
EXPENSE_RATIO = 0.40        # 40% of revenue goes to Fees, Cleaning, Taxes, Maintenance

# Gross Revenue
df_ready['gross_annual_revenue'] = df_ready['predicted_nightly'] * 365 * OCCUPANCY_RATE

# Net Revenue (What you actually pocket)
df_ready['net_annual_revenue'] = df_ready['gross_annual_revenue'] * (1 - EXPENSE_RATIO)

# Yield Calculation (Cap Rate)
df_ready['yield_percentage'] = (df_ready['net_annual_revenue'] / df_ready['price']) * 100

# --- 4. FILTER ---
# A realistic "Great Deal" is between 8% and 25% yield. 
# Anything > 30% is usually a data error (price is too low or prediction too high).
df_final = df_ready[
    (df_ready['yield_percentage'] > 5) & 
    (df_ready['yield_percentage'] < 35)
].copy()

print(f"Prediction Complete.")
print(f"Found {len(df_final)} Realistic Arbitrage Opportunities (5% - 35% Yield).")

display(df_final[['clean_address', 'price', 'predicted_nightly', 'net_annual_revenue', 'yield_percentage']].sort_values(by='yield_percentage', ascending=False).head(10))

Training on 24437 listings (outliers removed)...


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run wise-whale-142 at: https://adb-983293358114278.18.azuredatabricks.net/ml/experiments/882120521370948/runs/f27e4ecbb7c241919850df506fb5bd64
🧪 View experiment at: https://adb-983293358114278.18.azuredatabricks.net/ml/experiments/882120521370948
Prediction Complete.
Found 674 Realistic Arbitrage Opportunities (5% - 35% Yield).


clean_address,price,predicted_nightly,net_annual_revenue,yield_percentage
"156-08 riverside drive west, 10032 manhattan, New York, USA",388000.0,1096.9106,132122.88176999998,34.05228911597938
"3671 hudson manor terrace, 10463 bronx, New York, USA",279000.0,772.567861114286,93055.79887121575,33.353332928751165
"80-16 155th avenue 97, ny, 11414 howard beach, New York, USA",209999.0,572.7638666666667,68989.40774000001,32.85225536312078
"10027 manhattan, New York, USA",310000.0,845.0970773666667,101791.94296881501,32.83611063510162
"160 cabrini boulevard unit# 62, 10033 new york (manhattan), New York, USA",575000.0,1540.222624,185519.8150608,32.264315662747826
"1053 e 13th street, 11230 brooklyn, New York, USA",285000.0,757.0407,91185.55231500001,31.994930636842106
"120 cabrini boulevard, 10033 manhattan, New York, USA",550000.0,1460.9419048000002,175970.45243316007,31.99462771512001
"11218 brooklyn, New York, USA",475000.0,1234.2555773809527,148666.08429553575,31.298123009586476
"785 hart 11237 ll25-22042, New York, USA",300000.0,772.2423095238098,93016.5861821429,31.005528727380966
"120 cabrini boulevard, 10033 manhattan, New York, USA",575000.0,1460.9419048000002,175970.45243316007,30.603556944897402


Visualization

In [0]:
# ---------------------------------------------------------
# VISUALIZE
# ---------------------------------------------------------
import plotly.express as px

fig = px.scatter_mapbox(
    df_final,
    lat="lat", lon="lng",
    color="yield_percentage",
    size="net_annual_revenue",
    hover_name="clean_address",
    # We add sqft here so the user can see it on the map
    hover_data=["price", "beds", "baths", "sqft", "is_luxury", "is_penthouse"],
    color_continuous_scale="RdYlGn",
    zoom=10, height=600,
    title="Arbitrage City: Yield Map (NYC)"
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

fig.write_html("/dbfs/FileStore/nyc_map_final.html")